In [1]:
import re
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import sys
sys.path.append('src/')

from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, Vocabulary, create_vocabulary

from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
# ignore some deprecation warnings
warnings.filterwarnings('ignore')

## Functions

In [2]:
def pad_sequence(tokenizer_array, desired_length):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

def preprocess_smiles_data(x):
    x = tk.tokenize(x)
    x = vocabulary.encode(x )
    x  = pad_sequence(x, 200 )
    x  = torch.tensor([x ])
    
    return X

## Load main data set

In [3]:
df = pd.read_csv('retrosynthesis-all', header=None)
df['source'] = df[0].apply(lambda x: x.split('>>')[0])
df['target'] = df[0].apply(lambda x: x.split('>>')[1])
df.drop(0, axis=1, inplace=True)

# Remove spaces from all columns
for col in df.columns:
    df[col] = df[col].str.replace(' ', '')

## Define `Tokenizer` and `Vocabulary`

In [94]:
class Dataset(torch.utils.data.Dataset):
    """
    Custom PyTorch Dataset that takes a file containing \n separated SMILES.
    """

    def __init__(self, smiles_list : list, vocabulary : Vocabulary,
                 tokenizer : SMILESTokenizer) -> None:
        self._vocabulary  = vocabulary
        self._tokenizer   = tokenizer
        self._smiles_list = list(smiles_list)

    def __getitem__(self, i : int) -> torch.Tensor:
        smi     = self._smiles_list[i]
        tokens  = self._tokenizer.tokenize(smi)
        encoded = self._vocabulary.encode(tokens)
        return torch.tensor(encoded.astype(int), dtype=torch.long)  # pylint: disable=E1102

    def __len__(self) -> int:
        return len(self._smiles_list)

    @staticmethod
    def collate_fn(encoded_seqs : list) -> torch.Tensor:
        """
        Converts a list of encoded sequences into a padded tensor.
        """
        max_length   = max([seq.size(0) for seq in encoded_seqs])
        collated_arr = torch.zeros(len(encoded_seqs),
                                   max_length,
                                   dtype=torch.long)  # padded with zeros
        for i, seq in enumerate(encoded_seqs):
            collated_arr[i, :seq.size(0)] = seq
        return collated_arr

In [250]:
# create a vocabulary using all SMILES in df
dataset = df['source'].unique().tolist() + df['target'].unique().tolist()
dataset = np.unique(dataset).tolist()

tokenizer = SMILESTokenizer()
vocabulary   = create_vocabulary(smiles_list=dataset, tokenizer=tokenizer, canonical=True)
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')

train_dataset = Dataset(smiles_list=df['source'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
train_dataset = train_dataset.collate_fn(train_dataset)

There are 85 unique tokens in the vocabulary.



In [252]:
train_dataset[22].unsqueeze(0).shape

torch.Size([1, 160])

In [88]:
 torch.tensor([[21, 21, 20]]).shape

torch.Size([1, 3])

## Train / validation / test split

In [256]:
from sklearn.model_selection import train_test_split

print(df.shape)

# Splitting the data into train and combined val/test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting the combined val/test set into separate val and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.2, random_state=42)

# Printing the sizes of the resulting splits
print("Train data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_data))

train     = train_data.copy()
test      = test_data.copy()
valid     = val_data.copy()

train_X = Dataset(smiles_list=train['source'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
train_X = train_X.collate_fn(train_X)
train_y = Dataset(smiles_list=train['target'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
train_y = train_y.collate_fn(train_y)

val_X = Dataset(smiles_list=train['source'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
val_X = val_X.collate_fn(val_X)
val_y = Dataset(smiles_list=train['target'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
val_y = val_y.collate_fn(val_y)

test_X = Dataset(smiles_list=train['source'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
test_X = test_X.collate_fn(test_X)
test_y = Dataset(smiles_list=train['target'].tolist(), vocabulary=vocabulary, tokenizer=tokenizer)
test_y = test_y.collate_fn(test_y)

(45033, 2)
Train data size: 36026
Validation data size: 7205
Test data size: 1802


## Model Building

In [295]:
class SmilesLSTM(torch.nn.Module):
    def __init__(self, vocab_size, emb_size, num_layers, dropout=0, hidden_layers=5):
        super(SmilesLSTM, self).__init__()
        self.hidden_layers = hidden_layers
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.num_layers = num_layers
        self.dropout = dropout
        # define models
        self._embedding = nn.Embedding(num_embeddings=self.vocab_size,
                               embedding_dim=self.emb_size)
        
        self._reccurent = nn.LSTM(input_size=self.emb_size,
                          hidden_size=self.hidden_layers,
                          num_layers=self.num_layers,
                          dropout=self.dropout,
                          batch_first=True)
         
        self._linear = nn.Linear(self.hidden_layers, self.vocab_size) # (batch size) x (sequence length) x (vocabulary size)

    def forward(self, x):
        x = x.unsqueeze(0)
        
        embedding = self._embedding(x)
        rnn_output, (_, _) = self._reccurent(embedding)
        linear = self._linear(rnn_output)
        log_softmax = linear.log_softmax(dim=2)
        output = log_softmax.argmax(dim=2).flatten().tolist()
        return log_softmax, output

In [296]:
model = SmilesLSTM(vocab_size=len(vocabulary), emb_size=5, num_layers=1)
log_softmax, output = model(train_X[99])
y_true_smiles = train['target'].iloc[99]
y_true_token = train_y[99].unsqueeze(0)

In [297]:
# for idx, (correct_token, most_probable_token) in enumerate(zip(train['source'].iloc[99], y_pred)):
#     print(f"At time step {idx+1}, the generative model proposes {vocabulary.tokens()[most_probable_token]} as the most probable token and the correct token is {correct_token}")

In [298]:
y_true_token.shape

torch.Size([1, 160])

In [299]:
# loss = torch.nn.NLLLoss(reduction='none')
# print(f"The output tensor from negative log-likelihood is:\n{loss(log_softmax.transpose(1, 2), y_true_token)}\n")
# loss(log_softmax.transpose(1, 2), y_true_token).sum(dim=1)

In [300]:
data_loader_X = torch.utils.data.DataLoader(train_X, batch_size=1, shuffle=True, collate_fn=Dataset.collate_fn)
data_loader_y = torch.utils.data.DataLoader(train_y, batch_size=1, shuffle=True, collate_fn=Dataset.collate_fn)

EPOCHS = 3
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.NLLLoss(reduction='none')

model.train()
for epoch in range(EPOCHS):
    for row in range(len(train_X)):
        X = train_X[row]
        y = train_y[row]
        log_softmax, y_pred = model(X)
        loss = loss_fn(log_softmax.transpose(1, 2), y.unsqueeze(0)).sum(dim=1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

## Evaluation

In [301]:
test.head(1)

,source,target
14443,CC(C)N(C(=O)C[C@H]1C[C@@H](C=O)OC(C)(C)O1)C(C)C,CC(C)N(C(=O)C[C@H]1C[C@@H](CO)OC(C)(C)O1)C(C)C


In [302]:
test_X[0], test_y[0]

(tensor([ 1, 20, 20, 25, 20,  3, 17, 25,  4, 81,  8, 82, 81,  3, 20, 20,  3, 20,
          4, 20,  4, 81,  9, 82, 81,  3,  5, 81, 10, 81, 81, 81, 81, 81, 10,  4,
         84, 81,  9, 81,  8, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 tensor([ 1, 20, 17, 20,  3, 20,  4, 20, 81,  8, 82, 81,  3, 20,  3, 17, 25,  4,
         25, 20, 20,  4, 81,  3, 25,  4, 81,  9, 84, 81,  3,  5, 81, 10, 81, 81,
         81, 81, 81, 10,  4, 82, 81,  8,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  

In [303]:
_, y_pred = model(test_X[0])
y_pred

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]